# Notebook requêtant wikidata en SPARQL

Imports

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import numpy as np
import pandas as pd
import folium
from folium.plugins import MarkerCluster

### 1) Récupération de toutes les entités administratives de Paris

In [2]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery("""
SELECT ?item ?itemLabel
WHERE
{
?item wdt:P131 wd:Q90 . #Tous les 'located in the administrative territorial entity' de Paris
?item wdt:P31 wd:Q702842 . #Toutes les instances de 'municipal arrondissement'

SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" }
}
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

Stock résultats

In [8]:
list_arr = []
list_entity = []
for result in results['results']['bindings']:
    #stock de tous les noms d'arrdt
    list_arr = np.append(list_arr, result['itemLabel']['value'])
    #strock de tous les IDs des arrdt
    entity = result['item']['value']
    entity = entity.replace("http://www.wikidata.org/entity/", "")
    list_entity = np.append(list_entity, entity)
    #Print
    print(result['itemLabel']['value'] + " - " + entity)

1st arrondissement of Paris - Q161741
10th arrondissement of Paris - Q163948
11th arrondissement of Paris - Q169293
12th arrondissement of Paris - Q171689
13th arrondissement of Paris - Q175129
14th arrondissement of Paris - Q187153
15th arrondissement of Paris - Q191066
16th arrondissement of Paris - Q194420
17th arrondissement of Paris - Q197297
18th arrondissement of Paris - Q200126
19th arrondissement of Paris - Q204622
2nd arrondissement of Paris - Q209549
20th arrondissement of Paris - Q210720
3rd arrondissement of Paris - Q223140
4th arrondissement of Paris - Q230127
5th arrondissement of Paris - Q238723
6th arrondissement of Paris - Q245546
7th arrondissement of Paris - Q259463
8th arrondissement of Paris - Q270230
9th arrondissement of Paris - Q275118


In [5]:
list_entity

array(['Q161741', 'Q163948', 'Q169293', 'Q171689', 'Q175129', 'Q187153',
       'Q191066', 'Q194420', 'Q197297', 'Q200126', 'Q204622', 'Q209549',
       'Q210720', 'Q223140', 'Q230127', 'Q238723', 'Q245546', 'Q259463',
       'Q270230', 'Q275118'], dtype='<U32')

Préparation de la requête SPARQL

In [12]:
str = ""
for r in list_entity:
    if str == "":
        str = str+"{?item wdt:P131 wd:"+r+" .}"
    else:
        str = str+" UNION {?item wdt:P131 wd:"+r+" .}"
str

'{?item wdt:P131 wd:Q161741 .} UNION {?item wdt:P131 wd:Q163948 .} UNION {?item wdt:P131 wd:Q169293 .} UNION {?item wdt:P131 wd:Q171689 .} UNION {?item wdt:P131 wd:Q175129 .} UNION {?item wdt:P131 wd:Q187153 .} UNION {?item wdt:P131 wd:Q191066 .} UNION {?item wdt:P131 wd:Q194420 .} UNION {?item wdt:P131 wd:Q197297 .} UNION {?item wdt:P131 wd:Q200126 .} UNION {?item wdt:P131 wd:Q204622 .} UNION {?item wdt:P131 wd:Q209549 .} UNION {?item wdt:P131 wd:Q210720 .} UNION {?item wdt:P131 wd:Q223140 .} UNION {?item wdt:P131 wd:Q230127 .} UNION {?item wdt:P131 wd:Q238723 .} UNION {?item wdt:P131 wd:Q245546 .} UNION {?item wdt:P131 wd:Q259463 .} UNION {?item wdt:P131 wd:Q270230 .} UNION {?item wdt:P131 wd:Q275118 .}'

### 2) Requête tous les monuments historiques de Paris

In [13]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery("""
SELECT ?item ?itemLabel ?lon ?lat
WHERE
{
?item wdt:P1435 wd:Q10387575 .
"""+str+"""
 ?item p:P625 ?coordinate.
 ?coordinate ps:P625 ?coord.
 ?coordinate psv:P625 ?coordinate_node.
 ?coordinate_node wikibase:geoLongitude ?lon.
 ?coordinate_node wikibase:geoLatitude ?lat.  

SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],fr" }
}
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()


Stock tous les monuments dans un tableau

In [14]:
monuments = []
for r in results['results']['bindings']:
    monument = {}
    monument['nom'] = r['itemLabel']['value']
    monument['lon'] = r['lon']['value']
    monument['lat'] = r['lat']['value']
    monuments.append(monument)

In [16]:
df = pd.DataFrame(monuments)
df

,nom,lon,lat
0,cour de l'Industrie,2.38714,48.8508
1,faïencerie Loebnitz,2.3712,48.8665
2,fontaine du Château d'eau,2.3922777777778,48.889555555556
3,fontaine du Château d'eau,2.363,48.8682
4,fontaine du Palmier,2.34726944,48.85749722
...,...,...,...
1581,hôtel de Villeroy,2.3047222222222,48.865277777778
1582,église Saint-Bernard de la Chapelle,2.355,48.886111111111
1583,théâtre des Deux Ânes,2.33167,48.8842
1584,église Saint-Christophe-de-Javel,2.2795,48.8445


### 3) Display on a map

In [17]:
m = folium.Map(location=[48.866667,2.333333], tiles="OpenStreetMap", zoom_start=12)

In [18]:
mc = MarkerCluster()

In [19]:
for index, row in df.iterrows():
    mc.add_child(folium.Marker(location=[row['lat'],row['lon']], popup='<i>'+row['nom']+'</i>',tooltip=row['nom']))

m.add_child(mc)
m